# Drifter Simmulation in the JDF
Based off of code from https://github.com/UBC-MOAD/PythonNotes/blob/master/OceanParcelsRecipes.ipynb

In [1]:
import numpy as np
import xarray as xr
import os
from matplotlib import pyplot as plt, animation
from datetime import datetime, timedelta
from dateutil.parser import parse
from IPython.display import HTML
from salishsea_tools import nc_tools, places

from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, AdvectionRK4, AdvectionRK4_3D

%matplotlib inline

INFO: Compiled ParcelsRandom ==> /tmp/parcels-2919/libparcels_random_97c5f831-67b8-4b05-86ca-a7e57d00ab46.so


In [2]:
plt.rcParams['font.size'] = 12

#### Define Fieldset Functions
The following functions handle the syntax of defining the forcing files into a parcels.FieldSet object - get it into the right format for OceanParcels to load forcing data on-the-fly and handle all of the interpolation

In [3]:
def fieldset_from_nemo(daterange, coords, flat=True):
    """Generate a fieldset from a hourly SalishSeaCast forcing fields
    over daterange.
    """

    # Generate sequential list of forcing file prefixes
    prefixes = [
        nc_tools.get_hindcast_prefix(daterange[0] + timedelta(days=d))
        for d in range(np.diff(daterange)[0].days + 1)
    ]

    # Predefine fieldset argument dictionaries
    filenames, variables, dimensions = {}, {}, {}

    # Define dict fields for each variable
    for var, name in zip(['U', 'V', 'W'], ['vozocrtx', 'vomecrty', 'vovecrtz']):
        
        # Exclude vertical velocity if 2D
        if flat:
            if var == 'W': break

        # Dict of filenames containing the coordinate and forcing variables
        datafiles = [prefix + f'_grid_{var}.nc' for prefix in prefixes]
        filenames[var] = {'lon': coords, 'lat': coords, 'data': datafiles}

        # NEMO variable name
        variables[var] = name

        # Dict of NEMO coordinate names (f-points)
        dimensions[var] = {'lon': 'glamf', 'lat': 'gphif', 'time': 'time_counter'}
        
        # Add depth fields if 3D (f-points are on W grid)
        if not flat:
            filenames[var]['depth'] = prefixes[0] + '_grid_W.nc'
            dimensions[var]['depth'] = 'depthw'

    # Load NEMO forcing into fieldset
    field_set = FieldSet.from_nemo(filenames, variables, dimensions, field_chunksize='auto')
    
    return field_set


def append_auxiliary_forcing(fieldset, model, daterange, coords_path):
    """Append hourly GEM or WW3 forcing fields to a fieldset over daterange
    """

    # Generate sequential list of forcing files
    datafiles = [
        getattr(nc_tools, f'get_{model}_path')(daterange[0] + timedelta(days=d))
        for d in range(np.diff(daterange)[0].days + 1)
    ]
    
    # Assign variable suffix and dimensions definitions according to model
    if model == 'GEM':
        suffix = '_wind'
        dimensions = {'lon': 'nav_lon', 'lat': 'nav_lat', 'time': 'time_counter'}
    elif model == 'WW3':
        suffix = 'uss'
        dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
    else:
        raise ValueError(f'Unknown surface forcing model: {model}')
    
    # Define coordinates file path and create if necessary
    #  - only method I've had success with for correcting lons
    coords = os.path.join(coords_path, f'{model}_grid.nc')
    if not os.path.exists(coords):
        with xr.open_dataset(datafiles[0]) as ds:
            data_vars = [dimensions['time']] + list(ds.data_vars)
            if model == 'GEM':
                for key in ['lon', 'lat']: data_vars.remove(dimensions[key])
            ds = ds.drop_vars(data_vars)
            ds.update({dimensions['lon']: ds[dimensions['lon']] - 360})
            ds.to_netcdf(coords)
    
    # Filenames dict
    filenames = {'lon': coords, 'lat': coords, 'data': datafiles}
    
    # Load u velocity
    u = Field.from_netcdf(
        filenames, f'u{suffix}', dimensions, fieldtype='U', field_chunksize='auto',
    )
    
    # Load v velocity with u grid
    v = Field.from_netcdf(
        filenames, f'v{suffix}', dimensions, fieldtype='V', field_chunksize='auto',
        grid=u.grid, dataFiles=u.dataFiles,
    )

    # Add velocity fields to fieldset
    fieldset.add_field(u)
    fieldset.add_field(v)

    # Make new vector field from u and v for use in kernels
    u, v = [getattr(fieldset, f'{var}{suffix}') for var in ('u', 'v')]
    fieldset.add_vector_field(VectorField(f"UV_{model}", u, v))

#### Define Kernel Functions
Used to prescribe particle behavior in OceanParcels

In [4]:
def WindDrift(particle, fieldset, time):
    (uw, vw) = fieldset.UV_GEM[time, particle.depth, particle.lat, particle.lon]
    particle.lon += uw * 0.03 * particle.dt
    particle.lat += vw * 0.03 * particle.dt


def StokesDrift(particle, fieldset, time):
    (us, vs) = fieldset.UV_WW3[time, particle.depth, particle.lat, particle.lon]
    particle.lon += us * particle.dt
    particle.lat += vs * particle.dt


def DeleteParticle(particle, fieldset, time): #Susan had this one in her code but not the others
    print(f'Particle {particle.id} lost !! [{particle.lon}, {particle.lat}, {particle.depth}, {particle.time}]')
    particle.delete()

### Simulation

In [5]:
# Paths and filenames
paths = {
    'coords': '/ocean/rbeutel/MEOPAR/grid/coordinates_seagrid_SalishSea201702.nc',
    'mask': '/ocean/rbeutel/MEOPAR/grid/mesh_mask201702.nc',
    'results':  './results',
}

# Load coords and mask files and extract grid variables
coords, mask = [xr.open_dataset(paths[key], decode_times=False) for key in ('coords', 'mask')]
gridlon, gridlat = [coords[key][0, ...].values for key in ('glamt', 'gphit')]
tmask = mask.tmask[0, 0, ...].values

In [6]:
# Define release parameters
location = 'Juan de Fuca Strait'
n = 100   # number of particles
r = 50   # radius of particle cloud [m]

# Create Gaussian distribution around release point
mean, cov = [0, 0], [[r**2, 0], [0, r**2]]
x_offset, y_offset = np.random.multivariate_normal(mean, cov, n).T
lon, lat = places.PLACES[location]['lon lat']
lons = lon + x_offset / 111000 / np.cos(np.deg2rad(lat))
lats = lat + y_offset / 111000

In [7]:
# Start time, duration and timestep
start = datetime(2019, 1, 1, 12, 30, 0)
duration = timedelta(days=3)
dt = timedelta(seconds=90)

In [8]:
# Forcing daterange (I add 1-day buffers)
daterange = [start - timedelta(days=1), start + duration + timedelta(days=1)]

In [9]:
# Output file prefix
strings = [location] + [t.strftime('%Y%m%dT%H%M%S') for t in (start, start + duration)]
prefix = os.path.join(paths['results'], '_'.join(strings))

### 2D Surface Drift Simmulation

In [10]:
# Load SalishSeaCast results into fieldset
fieldset = fieldset_from_nemo(daterange, paths['coords'])

# Append GEM and WW3 results into fieldset
for model in ['GEM', 'WW3']:
    append_auxiliary_forcing(fieldset, model, daterange, paths['results'])

         It will be opened with no decoding. Filling values might be wrongly parsed.


SyntaxError: Field received an unexpected keyword argument "field_chunksize" (<string>)

In [13]:
# Execute NEMO-only, 2D run
pset = ParticleSet.from_list(fieldset, JITParticle, lon=lons, lat=lats, time=np.repeat(start, n))
kernel = AdvectionRK4
output_file = pset.ParticleFile(name=prefix + '_NEMO_2D.nc', outputdt=timedelta(hours=1))
pset.execute(
    kernel, runtime=duration, dt=dt, output_file=output_file,
    recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
)

NameError: name 'fieldset' is not defined